In [50]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import getpass 
import configparser

In [52]:
parser=configparser.ConfigParser()
parser.read('nb.cfg.txt')

['nb.cfg.txt']

In [54]:
conn_string=parser.get('my_db','connection_string')

engine= create_engine(conn_string)

In [60]:
with engine.connect() as connection:
    query = text('SELECT * FROM rental')
    result = connection.execute(query)
    df = pd.DataFrame(result.all())

df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [88]:
def rentals_month(engine,month,year):
    """retrieves rental data for a given month and year"""
    with engine.connect() as connection:
        query = text("SELECT *, MONTH(rental_date) AS rental_month, YEAR(rental_date) AS rental_year FROM rental WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year")
        result = connection.execute(query,{"month": month, "year": year})
        df = pd.DataFrame(result.all())
    return df
 
rentals_month(engine,6,2005)


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,rental_month,rental_year
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53,6,2005
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53,6,2005
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53,6,2005
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53,6,2005
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53,6,2005
...,...,...,...,...,...,...,...,...,...
2306,3465,2005-06-21 22:10:01,1488,510,2005-06-30 21:35:01,1,2006-02-15 21:30:53,6,2005
2307,3466,2005-06-21 22:13:33,371,226,2005-06-25 21:01:33,2,2006-02-15 21:30:53,6,2005
2308,3467,2005-06-21 22:19:25,729,543,2005-06-27 00:03:25,2,2006-02-15 21:30:53,6,2005
2309,3468,2005-06-21 22:43:45,2899,100,2005-06-30 01:49:45,1,2006-02-15 21:30:53,6,2005


In [106]:
def rental_count_month(engine,month,year):
    """takes the DataFrame provided by rentals_month as input along with the month and year 
    and returns a new DataFrame containing the number of rentals made by 
    each customer_id during the selected month and year"""
    with engine.connect() as connection:
        connection.execute(text("DROP TEMPORARY TABLE IF EXISTS month_year_rental"))
        query1 = text("CREATE TEMPORARY TABLE month_year_rental SELECT rental_id, customer_id, rental_date, MONTH(rental_date) AS rental_month, YEAR(rental_date) AS rental_year FROM rental WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year")
        query2= text("SELECT myr.customer_id, COUNT(r.rental_id) FROM month_year_rental AS myr JOIN rental AS r ON myr.customer_id = r.customer_id GROUP BY myr.customer_id")
        result = connection.execute(query1,{"month": month, "year": year})
        result2= connection.execute(query2)
        rental_count_df = pd.DataFrame(result2.all())
    return rental_count_df

rental_count_month(engine,6,2005)


,customer_id,COUNT(r.rental_id)
0,416,155
1,516,156
2,239,170
3,285,78
4,310,120
...,...,...
585,412,21
586,335,23
587,226,42
588,22,22


In [139]:
def compare_rentals(engine,month1,year1,month2,year2):
    """returns a combined DataFrame with a new 'difference' column, 
    which is the difference between the number of rentals in the two months"""
    with engine.connect() as connection:
        connection.execute(text("DROP TEMPORARY TABLE IF EXISTS month_year_rental"))
        connection.execute(text("DROP TEMPORARY TABLE IF EXISTS month_year_rental2"))
        connection.execute(text("DROP TEMPORARY TABLE IF EXISTS rental_count_month"))
        connection.execute(text("DROP TEMPORARY TABLE IF EXISTS rental_count_month2"))
        query1 = text("CREATE TEMPORARY TABLE month_year_rental SELECT rental_id, customer_id, rental_date, MONTH(rental_date) AS rental_month, YEAR(rental_date) AS rental_year FROM rental WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year")
        query2 = text("CREATE TEMPORARY TABLE month_year_rental2 SELECT rental_id, customer_id, rental_date, MONTH(rental_date) AS rental_month, YEAR(rental_date) AS rental_year FROM rental WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year")
        query3 = text("CREATE TEMPORARY TABLE rental_count_month AS SELECT customer_id, COUNT(rental_id) AS count1 FROM month_year_rental GROUP BY customer_id")
        query4 = text("CREATE TEMPORARY TABLE rental_count_month2 AS SELECT customer_id, COUNT(rental_id) AS count2 FROM month_year_rental2 GROUP BY customer_id")
        query5 = text("""
    SELECT rcm.customer_id, 
           COALESCE(rcm.count1, 0) AS count1, 
           COALESCE(rcm2.count2, 0) AS count2, 
           COALESCE(rcm.count1, 0) - COALESCE(rcm2.count2, 0) AS difference
    FROM rental_count_month AS rcm
    LEFT JOIN rental_count_month2 AS rcm2 
    ON rcm.customer_id = rcm2.customer_id
    UNION
    SELECT rcm2.customer_id, 
           COALESCE(rcm.count1, 0) AS count1, 
           COALESCE(rcm2.count2, 0) AS count2, 
           COALESCE(rcm.count1, 0) - COALESCE(rcm2.count2, 0) AS difference
    FROM rental_count_month2 AS rcm2
    LEFT JOIN rental_count_month AS rcm 
    ON rcm2.customer_id = rcm.customer_id
""""")
        connection.execute(query1,{"month": month1, "year": year1})
        connection.execute(query2,{"month": month2, "year": year2})
        connection.execute(query3)
        connection.execute(query4)
        result=connection.execute(query5)
        rental_count_df = pd.DataFrame(result.all(),columns=["customer_id", "count1", "count2", "difference"])
    return rental_count_df

compare_rentals(engine,5,2005,6,2005)


OperationalError: (pymysql.err.OperationalError) (1137, "Can't reopen table: 'rcm2'")
[SQL: 
    SELECT rcm.customer_id, 
           COALESCE(rcm.count1, 0) AS count1, 
           COALESCE(rcm2.count2, 0) AS count2, 
           COALESCE(rcm.count1, 0) - COALESCE(rcm2.count2, 0) AS difference
    FROM rental_count_month AS rcm
    LEFT JOIN rental_count_month2 AS rcm2 
    ON rcm.customer_id = rcm2.customer_id
    UNION
    SELECT rcm2.customer_id, 
           COALESCE(rcm.count1, 0) AS count1, 
           COALESCE(rcm2.count2, 0) AS count2, 
           COALESCE(rcm.count1, 0) - COALESCE(rcm2.count2, 0) AS difference
    FROM rental_count_month2 AS rcm2
    LEFT JOIN rental_count_month AS rcm 
    ON rcm2.customer_id = rcm.customer_id
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)